# Probablistic Model

In [2]:
import math
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn import preprocessing
from Training import model, utils, dataset, train
from sklearn.model_selection import train_test_split

import keras.backend as K

ModuleNotFoundError: No module named 'keras'

In [ ]:
def check_acc(y_hat,y,margin=0.05):
    a_err = (np.abs(y_hat - y)) # get normalized error 
    err = np.divide(a_err, y, out=a_err, where=y==0)
    assert(err.shape == y.shape)
    num_correct = 0
    for row in err:
        num_in_row = len(np.where(row < margin)[0]) # margin * 100 because 
        if num_in_row == len(row):
            num_correct += 1

    num_samples = y.shape[0]
    correct_idx = np.where(err < margin)
    num_part_correct = len(correct_idx[0])
    num_part_samples = y.shape[0] * y.shape[1]
    print(f"Correct = {num_correct} / {num_samples}")
    return (num_correct/num_samples)

#### Load Data

In [ ]:
data = utils.parseGainAndBWCsv2("Data/BW-3000.csv").astype(float)

In [ ]:
print(data[0])

##### Split data into inputs X and target Y Normalize to [0,1] (inclusive) and create Training and Testign Splits


In [ ]:
data = preprocessing.MinMaxScaler((0,1)).fit_transform(data)
X = data[:,:2]
Y = data[:,2:]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

#### Define Model and Parameters

##### Define model

* Model: MLP with variable hidden layer number and width  
* Output layer: 
    * Node 1 and 2 are the mean for probability mean 
    * Node 3 and 4 are the diagonal of the covariance matrix


In [ ]:
# num_hidden_layers = 5
# hidden_layer_size = 500

# model = tf.keras.models.Sequential([])

# for i in range(num_hidden_layers):
#     model.add(tf.keras.layers.Dense(hidden_layer_size, activation='relu'))
# model.add(tf.keras.layers.Dense(4, activation='sigmoid')) # add output layer: node 1,2 are 

In [ ]:
test_model = model.Model500RELUBN(2,4)

##### Define Loss

* Loss: guassian_nll
    * cite: https://gist.github.com/sergeyprokudin/4a50bf9b75e0559c1fcd2cae860b879e

In [ ]:
optimizer = optim.Adam(test_model.parameters(),lr=0.01)
loss_fn = nn.GaussianNLLLoss()

dataset1 = dataset.CircuitSynthesisGainAndBandwidthManually(Y, X)
train_dataset, val_dataset = utils.splitDataset(dataset1, 0.8)
    
train_data = DataLoader(train_dataset,batch_size = 500)
validation_data = DataLoader(val_dataset, batch_size = 500)

In [ ]:
epochs = 1000
losses, accs, part_accs = train.trainProbModel(test_model, train_data, loss_fn, optimizer, num_epochs=epochs, print_every=10)

## Testing scipt

My idea of the testing script. 
It takes the inverse simulator prediction (Y -> X^)  
Pass X^ to mock_simulator to get Y^   
evaluate performance from Y to Y^  
I don't know if the data is being shuffled at all could contribute too the poor performance

In [ ]:
x_preds = test_model(torch.Tensor(Y))
#print(x_preds)
mock_simulator = tf.keras.models.load_model('mock_simulator')

final_preds = []
for i in range(x_preds.shape[0]):
    means = x_preds.detach().numpy()[i,:2].T
    var = x_preds.detach().numpy()[i,2:].T

    varM = np.zeros((2,2))
    np.fill_diagonal(varM,var)

    pts = np.random.default_rng().multivariate_normal(means, varM, 1).T # get predicted X value 
    y_hat = mock_simulator(pts.T) # feed back through simulator to get evaluated performance
    final_preds.append(y_hat)
final_preds = np.array(final_preds).reshape(-1,2)
    

# for i,d in enumerate(final_preds):
#     print(Y[i],d)
print(final_preds)


In [ ]:
check_acc(Y,final_preds,margin=.05)

In [ ]:
def gaussian_nll(ytrue, ypreds):
    """Keras implmementation of multivariate Gaussian negative loglikelihood loss function. 
    This implementation implies diagonal covariance matrix.
    
    Parameters
    ----------
    ytrue: tf.tensor of shape [n_samples, n_dims]
        ground truth values
    ypreds: tf.tensor of shape [n_samples, n_dims*2]
        predicted mu and logsigma values (e.g. by your neural network)
        
    Returns
    -------
    neg_log_likelihood: float
        negative loglikelihood averaged over samples
        
    This loss can then be used as a target loss for any keras model, e.g.:
        model.compile(loss=gaussian_nll, optimizer='Adam') 
    
    """
    print(f"ytrue: {ytrue.shape}")
    print(f"ypreds: {ypreds.shape}")
    n_dims = int(int(ypreds.shape[1])/2)
    mu = ypreds[:, 0:n_dims]
    logsigma = ypreds[:, n_dims:]
    sigma = ypreds[:, n_dims:]
    
    if K.any(sigma < 0):
        print(sigma)
        raise ValueError("sigma has negative entry/entries")
        
    #mse = -0.5*K.sum(K.square((ytrue-mu)/K.exp(logsigma)),axis=1)
    #sigma_trace = -K.sum(logsigma, axis=1)
    #log2pi = -0.5*n_dims*np.log(2*np.pi)
    
    log_likelihood = 0.5 * K.log(sigma**2) + (ytrue-mu)**2 / (sigma)

    return K.mean(log_likelihood)

In [ ]:
# loss_fn = gaussian_nll

# optim = tf.keras.optimizers.Adam(
#     learning_rate=0.001)
# model.compile(optimizer=optim,
#               loss=loss_fn)

In [ ]:
# model.fit(X_train, y_train, epochs=1000, batch_size= 700, validation_data=(X_test, y_test))